**전체적인 과정**
1. 데이터 전처리
- Family Size 에서 이상치 제거 (21 이상) (테스트 세트에 20까지 존재)
- Index Column 제거
- Correl 이용해 거꾸로 된 Q_A Score들 돌려주기 --> Mach Score 추출
- one-hot < label< automl 자체 처리 순으로 효율이 좋지 때문에 Categorical 변수는 그대로 유지

2. 모델 학습 (AUTO ML)
- 모델 학습 비교 (대부분 GBC LGBM LDA 가 좋게 나왔음)
- 모델 블렌딩
- Voting 통해 모델 확정

3. 다른 경우의 수
- 아무 전처리 없이 index 만 제거 + label->0.7654, 0.7614, 0.7614 (top 3)
- label 미진행 -> 0.7656, 0.7650, 0.7631
- label 미진행 + Q_A Score 거꾸로 + Mach Score--> 0.7669, 0.7663, 0.7629 (**아직까지는 최고 점수**)


In [ ]:
#필요한 모듈들 import
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv("/content/drive/Shareddrives/2022-KUBIG CONTEST_머신러닝 4팀/train.csv")
test=pd.read_csv("/content/drive/Shareddrives/2022-KUBIG CONTEST_머신러닝 4팀/test_x.csv")

submission=pd.read_csv("/content/drive/Shareddrives/2022-KUBIG CONTEST_머신러닝 4팀/sample_submission.csv", index_col=0)
print(train.shape)
print(test.shape)
print(submission.shape)
#데이터 불러오기

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(45532, 78)
(11383, 77)
(11383, 1)


In [ ]:
#pycaret 을 사용하기 위해 필요한 것들
!pip install markupsafe==2.0.1
!pip install Jinja2




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 412, in run
    env = get_environment(lib_locations)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/metadata/__init__.py", line 29, in get_environment
    return Environment.from_paths(paths)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/metadata/pkg_resources.py", line 84, in from_paths
    return cls(pkg_resources.WorkingSet(paths))
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 567, i

In [ ]:
import jinja2

ImportError: ignored

In [ ]:
!pip install pycaret[full]
#pycaret 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv("/content/drive/Shareddrives/2022-KUBIG CONTEST_머신러닝 4팀/train.csv")
test=pd.read_csv("/content/drive/Shareddrives/2022-KUBIG CONTEST_머신러닝 4팀/test_x.csv")

submission=pd.read_csv("/content/drive/Shareddrives/2022-KUBIG CONTEST_머신러닝 4팀/sample_submission.csv", index_col=0)
print(train.shape)
print(test.shape)
print(submission.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(45532, 78)
(11383, 77)
(11383, 1)


In [ ]:
pd.set_option('display.max_columns', None) #dataframe 한 번에 다 볼수 있게

In [ ]:
dataset = [train,test] 

In [ ]:
#주석 쳐져 있는 코드들은 진행해보고 성능이 좋지 않아 안 하는 것들
#from sklearn.preprocessing import LabelEncoder

In [ ]:
'''
#labelencoding
label = LabelEncoder()
train['gender_code'] = label.fit_transform(train['gender'])
train['age_group_code'] = label.fit_transform(train['age_group'])
train['race_code'] = label.fit_transform(train['race'])
train['religion_code'] = label.fit_transform(train['religion'])

test['gender_code'] = label.fit_transform(test['gender'])
test['age_group_code'] = label.fit_transform(test['age_group'])
test['race_code'] = label.fit_transform(test['race'])
test['religion_code'] = label.fit_transform(test['religion'])

drop2 = ['gender','age_group','race','religion']
train = train.drop(drop2, axis = 1)
test = test.drop(drop2, axis = 1)
'''

"\n#labelencoding\nlabel = LabelEncoder()\ntrain['gender_code'] = label.fit_transform(train['gender'])\ntrain['age_group_code'] = label.fit_transform(train['age_group'])\ntrain['race_code'] = label.fit_transform(train['race'])\ntrain['religion_code'] = label.fit_transform(train['religion'])\n\ntest['gender_code'] = label.fit_transform(test['gender'])\ntest['age_group_code'] = label.fit_transform(test['age_group'])\ntest['race_code'] = label.fit_transform(test['race'])\ntest['religion_code'] = label.fit_transform(test['religion'])\n\ndrop2 = ['gender','age_group','race','religion']\ntrain = train.drop(drop2, axis = 1)\ntest = test.drop(drop2, axis = 1)\n"

In [ ]:
'''
train = pd.get_dummies(data = train, columns = ['race','religion','age_group','gender'])
'''

"\ntrain = pd.get_dummies(data = train, columns = ['race','religion','age_group','gender'])\n"

In [ ]:
'''
test = pd.get_dummies(data = test,columns = ['race','religion','age_group','gender'])
'''

"\ntest = pd.get_dummies(data = test,columns = ['race','religion','age_group','gender'])\n"

In [ ]:
# train = train[train['familysize'] < 21]


In [ ]:
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for data in dataset:
  for flip in flipping_columns: 
    data[flip] = 6 - data[flip]


flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for data in dataset:
  for flip in flipping_secret_columns: 
    data[flip] = 6 - data[flip]

In [ ]:
answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

In [ ]:
# for data in dataset:
#   data['Mach_score'] = data[answers].mean(axis = 1)

In [ ]:
dataset[0]

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,30s,2,1,4,Female,1,3,White,Other,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
1,1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,20s,4,2,3,Female,1,1,Asian,Hindu,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1
2,2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,30s,3,1,3,Male,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
3,3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,20s,4,2,0,Female,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
4,4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,20s,3,1,2,Male,1,2,White,Agnostic,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,45527,2.0,1050,5.0,619,4.0,328,1.0,285,1.0,602,1.0,267,1.0,315,1.0,483,1.0,1016,4.0,278,4.0,611,5.0,215,4.0,388,1.0,299,2.0,1915,4.0,439,2.0,823,1.0,309,5.0,586,4.0,2252,10s,2,1,1,Female,3,1,White,Jewish,1,2,6,1,2,6,3,0,6,4,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
45528,45528,2.0,581,3.0,1353,4.0,1164,1.0,798,3.0,1680,2.0,560,2.0,640,1.0,1415,2.0,4494,5.0,1392,2.0,2478,5.0,1268,3.0,843,2.0,1401,4.0,1524,4.0,719,2.0,754,3.0,1118,4.0,654,4.0,1197,30s,2,1,2,Male,1,1,White,Atheist,2,2,2,3,1,3,3,4,1,5,1,2,0,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1
45529,45529,4.0,593,1.0,857,1.0,1047,4.0,1515,5.0,1690,4.0,1253,5.0,1094,1.0,1283,5.0,2209,1.0,1764,5.0,1062,5.0,1489,1.0,1908,4.0,891,1.0,1298,1.0,1200,5.0,473,5.0,1779,3.0,1355,1.0,667,10s,2,1,1,Female,1,1,White,Christian_Other,4,4,2,4,0,1,0,2,1,6,2,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1
45530,45530,1.0,747,3.0,1331,4.0,892,2.0,1281,1.0,1328,1.0,599,1.0,650,1.0,1429,5.0,1748,1.0,770,5.0,1025,5.0,742,4.0,4180,3.0,707,1.0,489,1.0,913,4.0,1301,4.0,1680,4.0,737,1.0,1124,40s,4,1,2,Male,3,1,White,Atheist,1,2,0,4,0,1,3,2,2,4,0,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,1,1


In [ ]:
for data in dataset:
  data.drop('index',axis=1,inplace = True)

In [ ]:
dataset[0].shape

(45532, 77)

In [ ]:
from pycaret.classification import *
clf = setup(
    data = dataset[0],
    target = 'voted',
    n_jobs=1
)

best_3 = compare_models(include=['lightgbm', 'catboost', 'lda', 'gbc'], sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6949,0.7655,0.7669,0.6383,0.6967,0.3949,0.4017,30.918
lightgbm,Light Gradient Boosting Machine,0.6940,0.7646,0.7588,0.6392,0.6938,0.3925,0.3983,1.611
catboost,CatBoost Classifier,0.6952,0.7646,0.7436,0.6443,0.6904,0.3933,0.3972,22.787
lda,Linear Discriminant Analysis,0.6931,0.7623,0.7373,0.6433,0.6871,0.3887,0.3922,1.511


INFO:logs:create_model_container: 4
INFO:logs:master_model_container: 4
INFO:logs:display_container: 2
INFO:logs:[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=3411, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, 

In [ ]:
print(best_3)

[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=3411, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=1, num_leaves=31, objective=N

In [ ]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7013,0.7741,0.7652,0.6463,0.7007,0.4069,0.4127
1,0.6913,0.7615,0.7642,0.6347,0.6935,0.3879,0.3947
2,0.6925,0.7634,0.7532,0.6387,0.6912,0.3891,0.3943
3,0.7022,0.7719,0.7545,0.6500,0.6983,0.4076,0.4120
4,0.6916,0.7623,0.7514,0.6379,0.6900,0.3872,0.3923
Mean,0.6958,0.7667,0.7577,0.6415,0.6948,0.3957,0.4012
Std,0.0049,0.0053,0.0058,0.0057,0.0041,0.0094,0.0091


INFO:logs:create_model_container: 5
INFO:logs:master_model_container: 5
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
          

In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                

In [ ]:
predictions = predict_model(final_model, data = test, raw_score=True)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                  

In [ ]:
predictions

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,Label,Score_0,Score_1
0,3.0,736,2.0,2941,3.0,4621,1.0,4857,2.0,2550,...,0,1,0,1,0,1,1,2,0.3193,0.6807
1,3.0,514,2.0,1952,3.0,1552,3.0,821,4.0,1150,...,0,0,0,0,0,0,0,2,0.1402,0.8598
2,3.0,500,2.0,2507,4.0,480,2.0,614,2.0,1326,...,1,1,0,1,0,1,1,1,0.5100,0.4900
3,1.0,669,1.0,1050,5.0,1435,2.0,2252,5.0,2533,...,1,1,1,1,1,1,1,1,0.7923,0.2077
4,2.0,499,1.0,1243,5.0,845,2.0,1666,2.0,925,...,1,1,0,1,1,1,1,2,0.2526,0.7474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,5.0,427,5.0,1066,5.0,588,1.0,560,2.0,1110,...,1,1,0,1,0,1,1,1,0.6145,0.3855
11379,1.0,314,5.0,554,5.0,230,1.0,956,2.0,1173,...,1,1,1,1,1,1,1,2,0.1298,0.8702
11380,1.0,627,2.0,799,1.0,739,2.0,1123,1.0,829,...,1,1,0,1,0,1,1,1,0.7623,0.2377
11381,2.0,539,1.0,2090,2.0,4642,1.0,673,2.0,1185,...,1,1,0,1,1,1,0,1,0.6868,0.3132


In [ ]:
submission['voted'] = predictions['Score_1']

In [ ]:
submission

,voted
index,
0,0.6807
1,0.8598
2,0.4900
3,0.2077
4,0.7474
...,...
11378,0.3855
11379,0.8702
11380,0.2377


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission_result_08302008.csv')